In [3]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense , Conv2D , MaxPooling2D , Flatten , Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [4]:
#  generator
train_ds = keras.utils.image_dataset_from_directory(
    directory = "OBJECTS/train",
    labels="inferred",
    label_mode="int",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(256, 256),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
    pad_to_aspect_ratio=False,
    data_format=None,
    verbose=True,
)

test_ds = keras.utils.image_dataset_from_directory(
    directory = "OBJECTS/test",
    labels="inferred",
    label_mode="int",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(256, 256),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
    pad_to_aspect_ratio=False,
    data_format=None,
    verbose=True,
)

Found 2527 files belonging to 6 classes.
Found 610 files belonging to 6 classes.


In [32]:
#  data augmenting 

train_datagen = ImageDataGenerator(
    rescale=1.0/255,  # Normalize the images
    rotation_range=40,  # Randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.2,  # Randomly translate images horizontally (fraction of total width)
    height_shift_range=0.2,  # Randomly translate images vertically (fraction of total height)
    shear_range=0.2,  # Randomly apply shearing transformations
    zoom_range=0.2,  # Randomly zoom in on images
    horizontal_flip=True,  # Randomly flip images horizontally
    fill_mode='nearest'  # Fill pixels that might have been lost after a transformation
)

validation_datagen = ImageDataGenerator(rescale=1.0/255)

# Create a data generator for the training data
train_generator = train_datagen.flow_from_directory(
    "OBJECTS/train",
    target_size=(256,256),  # Resize all images to 150x150
    batch_size=32,  # Number of images to yield per batch
    class_mode='categorical'  # Binary labels (use 'categorical' for multi-class labels)
)


Found 2527 images belonging to 6 classes.


In [5]:
# Normalize 
def Nomalize_image(image , label):
          image = tf.cast(image/255 , tf.float32)
          return image , label

train_ds = train_ds.map(Nomalize_image)

In [20]:
model = Sequential()
model.add(Conv2D(32 , kernel_size=(3,3)  , padding = "valid" , activation='relu' , input_shape = (256,256,3)))
model.add(MaxPooling2D(pool_size=(2,2) , strides=2 , padding="valid"))

model.add(Conv2D(64 , kernel_size=(3,3)  , padding = "valid" , activation='relu' ))
model.add(MaxPooling2D(pool_size=(2,2) , strides=2 , padding="valid"))

model.add(Conv2D(128 , kernel_size=(3,3)  , padding = "valid" , activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2) , strides=2 , padding="valid"))

model.add(Flatten())

model.add(Dense(128 , activation="relu"))
model.add(Dense(64 , activation="relu"))
model.add(Dense(1 , activation="softmax"))

In [21]:
model.compile(optimizer="rmsprop" , loss = "categorical_crossentropy" , metrics = ["accuracy"])

In [6]:
# Tuner 
import keras_tuner as kt

In [9]:
def build_model(hp):
    model = Sequential()
    count = 0

    # Convolutional layers
    for i in range(hp.Int("num_conv_layer", min_value=1, max_value=50)):
        if count == 0:
            model.add(Conv2D(
                filters=hp.Int('units' + str(i), min_value=8, max_value=128, step=8),
                kernel_size=(3, 3),
                activation=hp.Choice("activation" + str(i), values=["relu", "tanh", "sigmoid"]),
                padding="valid",
                input_shape=(256, 256, 3)
            ))
            model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding="valid"))
        else:
            model.add(Conv2D(
                filters=hp.Int('units' + str(i), min_value=8, max_value=128, step=8),
                kernel_size=(3, 3),
                activation=hp.Choice("activation" + str(i), values=["relu", "tanh", "sigmoid"]),
                padding="valid"
            ))
            model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding="valid"))
        count += 1

    model.add(Flatten())

    # Dense layers
    for i in range(hp.Int("num_dense_layer", min_value=1, max_value=50)):
        model.add(Dense(
            units=hp.Int("unit_dense" + str(i), min_value=8, max_value=128, step=8),
            activation=hp.Choice("activation_dense" + str(i), values=["relu", "tanh", "sigmoid"])
        ))
        model.add(Dropout(hp.Choice("dropout" + str(i), values=[0.2, 0.3, 0.4, 0.5])))

    model.add(Dense(1, activation="softmax"))
    model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])
    
    return model



           
                      

In [10]:
# Example usage with RandomSearch tuner
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='my_dir',
    project_name='helloworld'
)


tuner.search(train_ds , validation_data = test_ds , epochs=5)

# Retrieve the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=5)[0]

# Print the best hyperparameters
print(f"""
The hyperparameter search is complete. The optimal hyperparameters are:
- Number of convolutional layers: {best_hps.get('num_conv_layer')}
- Number of dense layers: {best_hps.get('num_dense_layer')}
- Other hyperparameters: {best_hps.values}
""")

# Build the model with the best hyperparameters
model = tuner.hypermodel.build(best_hps)



Reloading Tuner from my_dir\helloworld\tuner0.json

Search: Running Trial #4

Value             |Best Value So Far |Hyperparameter
2                 |2                 |num_conv_layer
8                 |32                |units0
tanh              |sigmoid           |activation0
3                 |2                 |num_dense_layer
40                |80                |unit_dense0
sigmoid           |tanh              |activation_dense0
0.4               |0.2               |dropout0
72                |24                |units1
relu              |tanh              |activation1
64                |24                |units2
sigmoid           |tanh              |activation2
64                |8                 |unit_dense1
tanh              |relu              |activation_dense1
0.2               |0.2               |dropout1
64                |40                |unit_dense2
sigmoid           |tanh              |activation_dense2
0.2               |0.4               |dropout2
56                

KeyboardInterrupt: 